In [1]:
import pandas as pd
import numpy as np

# Example 1 - consecutive rows

Create a synthetic time series dataset and calculate the "rolling difference"

In [2]:
df = pd.DataFrame()
df["date"] = pd.DatetimeIndex(start="20160101 00:00:00", end="20160131 23:00:00", freq="T")
df["measurement"] = np.random.randn(len(df))
print(len(df))
df.head()

44581


,date,measurement
0,2016-01-01 00:00:00,0.739159
1,2016-01-01 00:01:00,0.636203
2,2016-01-01 00:02:00,-1.746664
3,2016-01-01 00:03:00,-0.080402
4,2016-01-01 00:04:00,-1.312652


How do you get the difference from the previous value?

Naive option: loop through the dataset and for each row `i`, get the value with an index of `i+1` and subtract

In [3]:
def naive_diff(series):
    diff_values = []
    for i in range(len(series)):
        # first value needs to be NaN
        if i == 0:
            diff_values.append(np.NaN)
        else:
            diff_values.append(series[i] - series[i-1])
    return diff_values

df["diff"] = naive_diff(df["measurement"])
df.head()

,date,measurement,diff
0,2016-01-01 00:00:00,0.739159,NaN
1,2016-01-01 00:01:00,0.636203,-0.102955
2,2016-01-01 00:02:00,-1.746664,-2.382867
3,2016-01-01 00:03:00,-0.080402,1.666262
4,2016-01-01 00:04:00,-1.312652,-1.232250


That gives the correct value, but it feels slow even for ~44,000 rows

The better approach is to do a vectorised operation, with `.shift()`

In [4]:
def diff_with_shift(series):
    return series - series.shift()

df["diff_2"] = diff_with_shift(df["measurement"])

df.head()

,date,measurement,diff,diff_2
0,2016-01-01 00:00:00,0.739159,NaN,NaN
1,2016-01-01 00:01:00,0.636203,-0.102955,-0.102955
2,2016-01-01 00:02:00,-1.746664,-2.382867,-2.382867
3,2016-01-01 00:03:00,-0.080402,1.666262,1.666262
4,2016-01-01 00:04:00,-1.312652,-1.232250,-1.232250


Running that second method felt instantaneous - let's actually time the two approaches

In [5]:
%timeit naive_diff(df["measurement"])
%timeit diff_with_shift(df["measurement"])

1 loop, best of 3: 1.45 s per loop
1000 loops, best of 3: 667 µs per loop


Obviously the vectorised approach is **much faster**, that's what `numpy` and `pandas` are optimised for.

# Example 2 - date functions

What if we want to get just the day from the date?

Naive option: one by one

In [6]:
def naive_day(series):
    days = []
    for s in series:
        days.append(s.day)
    return days

Naive option 2: using `.apply()`

Vectorised option: using the in-built `.dt` accessor for date types

In [7]:
%timeit naive_day(df["date"])
%timeit df["date"].apply(lambda x: x.day)
%timeit df["date"].dt.day

1 loop, best of 3: 207 ms per loop
1 loop, best of 3: 180 ms per loop
100 loops, best of 3: 3.08 ms per loop
